In [1]:
import numpy as np
import pandas as pd

In [2]:
# manage space 
from dask.distributed import Client
import dask
client = Client(processes=False, threads_per_worker=1, n_workers=1, memory_limit='100GB')

In [5]:
# read the hit list of all fields
hit = pd.read_csv("hits.csv")
hit_li = hit["0"].values
hit_li = np.concatenate((['eid'], hit_li))

In [7]:
# check first phenotype field
path1 = "/u/project/sgss/UKBB/PRS-RESEARCH/03-compile-pheno/raw/ukb39967.enc_ukb.converted2.csv"
file1 = pd.read_csv(path1, usecols = hit_li, encoding = 'unicode_escape')
file1 = file1.set_index("eid")
file1.index.name = "eid"

In [10]:
# check second phenotype field
path2 = "/u/project/sgss/UKBB/PRS-RESEARCH/03-compile-pheno/raw/ukb21970.csv"
col_age = "21003-0.0"
col_sex = "31-0.0"
col_pcs = [f"22009-0.{i}" for i in range(1, 41)]
col_report_ethnic = "21000-0.0"
col_id = "eid"
col_dep = "189-0.0"

cols = [col_id, col_age, col_sex, col_report_ethnic, col_dep] + col_pcs
col_names = ["ID", "AGE", "SEX", "SIRE_ID", "DEPRIVATION_INDEX"] + [f"PC{i + 1}" for i in range(len(col_pcs))]

file2 = pd.read_csv(path2, usecols = cols, encoding = 'unicode_escape')[cols]
file2.columns = col_names
file2 = file2.set_index("ID")
file2.index.name = "eid"

In [12]:
# extract the common individual ids of these two files
# ends up 488266 indivs in total
idx = file1.index.intersection(file2.index)
file1 = file1.loc[idx]
file2 = file2.loc[idx]

In [15]:
file1.to_csv("pheno.csv", na_rep='NA')
file2.to_csv("covar.csv", na_rep='NA')